# Loading and Setup

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import re

# plotting
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import seaborn as sns

# stats
from statsmodels.api import tsa # time series analysis
import statsmodels.api as sm

import statsmodels.api as sm
!pip install pmdarima
from pmdarima.arima import auto_arima
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error
from sklearn import metrics


In [ ]:
maven_df = pd.read_pickle('Data/maven_toys.pkl')

# Assessment

In [ ]:
maven_df.tail()

In [ ]:
# Check the shape of the maven_dfFrame
maven_df.nunique()

In [ ]:
# Check the data types and missing values
maven_df.info()

In [ ]:
# Fill missing values in 'Customer_Satisfaction' with the median
# maven_df['Customer_Satisfaction'].fillna(maven_df['Customer_Satisfaction'].median(), inplace=True)

# Check if there are any missing values left
maven_df.isnull().sum()

In [ ]:
# Checking for duplicates
maven_df.duplicated().sum()



# Cleaning

In [ ]:
# Convert 'Date' and 'Store_Open_Date' to datetime type
maven_df['Date'] = pd.to_datetime(maven_df['Date'])
maven_df['Store_Open_Date'] = pd.to_datetime(maven_df['Store_Open_Date'])
maven_df['Product_Price'] = maven_df['Product_Price'].str.replace('$', '').astype(float)
maven_df['Product_Cost'] = maven_df['Product_Cost'].str.replace('$', '').astype(float)
maven_df['Product_Category'] = maven_df['Product_Category'].astype('category')
maven_df['Store_Location'] = maven_df['Store_Location'].astype('category')
maven_df['Customer_Reference'] = maven_df['Customer_Reference'].astype('category')
maven_df['Sale_ID'] = maven_df['Sale_ID'].astype(str)
maven_df['Store_ID'] = maven_df['Store_ID'].astype(str)
maven_df['Product_ID'] = maven_df['Product_ID'].astype(str)

# Extract the month and year from the 'Date' column
maven_df['YearMonth'] = maven_df['Date'].dt.to_period('M')

# Check the maven_df types again to confirm
maven_df.dtypes

## Feature Engineering

In [ ]:

maven_df['Price_Bins'] = pd.cut(maven_df['Product_Price'], bins=5, labels=['medium-low', 'low', 'medium', 'high', 'medium-high'])
maven_df['Product_profit'] = maven_df['Units'] * (maven_df['Product_Price'] - maven_df['Product_Cost'])

maven_df['Growth_Potential_1'] = maven_df.Product_profit * maven_df.Customer_Satisfaction
maven_df['Profit_Ratio'] = maven_df.Product_Price / maven_df.Product_Cost 
maven_df['Growth_Potential_2'] = maven_df.Profit_Ratio * maven_df.Customer_Satisfaction

maven_df




# EDA

In [ ]:
# Find the top 10 products with the highest customer satisfaction
maven_df.groupby('Product_Name')['Customer_Satisfaction'].describe().sort_values(by='mean',ascending=False).head(10)


In [ ]:
# Top 10 Stores with the highest customer satisfaction
maven_df.groupby('Product_Category')['Customer_Satisfaction'].describe().sort_values(by='mean',ascending=False).head(10)


In [ ]:

maven_df.groupby('Store_City')['Customer_Satisfaction'].describe().sort_values(by='mean',ascending=False).head(10)



In [ ]:
maven_df.groupby('Price_Bins')['Customer_Satisfaction'].describe().sort_values(by='mean',ascending=False).head(10)



In [ ]:
# Find the most popular product categories
maven_df.groupby('Price_Bins')['Customer_Satisfaction'].describe().sort_values(by='mean',ascending=False).head(10)



In [ ]:
# Compare the sales of the stores in Guadalajara to the sales of stores in other areas
maven_df.groupby('Product_Name')['Customer_Satisfaction'].describe().sort_values(by='mean',ascending=False).head(10)



In [ ]:
maven_df.groupby('Customer_Reference')['Customer_Satisfaction'].describe().sort_values(by='mean',ascending=False).head(10)


In [ ]:
# # Compare the sales of the individual stores in Guadalajara
maven_df.groupby('Product_Name')['Customer_Satisfaction'].describe().sort_values(by='mean',ascending=False).head(10)



In [ ]:
maven_df.groupby('Sale_Payment_Type')['Customer_Satisfaction'].mean().sort_values(ascending=False)

In [ ]:
# Calculate the average customer satisfaction
maven_df['Customer_Satisfaction'].hist(bins=10)


In [ ]:
topProduct = maven_df.groupby('Product_Name')[['Product_profit', 'Units']].sum().sort_values(by='Product_profit', ascending=False).head(10)
topProduct = topProduct.reset_index()

#Generating the labels which will show the sales value in K figure
topProduct['Total_Profit_M'] = ['$ '+str(round(int(v)/1000000,2))+' M' for v in topProduct.Product_profit]

topProduct


In [ ]:
topCategories = maven_df.groupby('Product_Category')[['Product_profit', 'Units']].agg(['mean', 'sum'])
topCategories = topCategories.reset_index()

#Generating the labels which will show the sales value in K figure
# topProduct['Total_Profit_M'] = ['$ '+str(round(int(v)/1000000,2))+' M' for v in topProduct.Product_profit]

topProduct


In [ ]:
# Calculate the monthly sales for each product category
category_sales = maven_df.groupby(['YearMonth', 'Product_Category'])['Customer_Satisfaction'].mean().unstack().rolling(3).mean().dropna()
category_sales


# Model Preparation, Fitting, and Testing

In [ ]:
dummy_df = pd.get_dummies(maven_df, 
        columns=['Store_Location', 'Product_Category', 'Sale_Payment_Type', 'Store_City', 'Customer_Reference'])

maven_df = pd.concat([maven_df, dummy_df], axis=1)




# Model Evaluation

# Final Insights

### How are we doing with customer satisfaction?

EDA: 
What areas have the highest customer satisfaction?  
Store Locations
City Locations

Model:
Linear Regression on Customer Satisfaction  
More Exploratory Data Analysis on Customer Satisfaction

### What does the data say about how we can grow our business?
Expansion: 
High Customer Satisfaction, Profit, and Amount of Units Sold

Time series on satisfaction, profit, units sold?

Independent
Products, Locations, Which Month, Store Open Date ---- Store Duration

Customer Reference

Target Category 
Profit and Satisfactory

1. Sum profit by Location and then Divide by Number of Days Open - Profitability Metric
   1. Store Location, Store City 
   2. Use this to evaluate the profitability of each store location
2. Products 
   1. Most profitable categories, Most profitable products
3. Which month is the most profitable
   1. Which month is the most profitable for each location







In [ ]:
# maven_df['Date'] = pd.to_datetime(maven_df['Date'])

# sales_weekly = pd.DataFrame(maven_df.groupby([maven_df['Date'].dt.year.rename('year'), 
#                 maven_df['Date'].dt.to_period("W").rename('week')])['Product_profit'].sum().reset_index().drop('year', axis=1).set_index('week'))

# sales_weekly
# arima_model=auto_arima(maven_df['Product_profit'], start_p=1,d=1,start_q=1,
#                 max_p=5,max_q=5,max_d=5,m=12,
#                 start_P=0,D=1,start_Q=0,max_P=5,max_D=5,max_Q=5,
#                 seasonal=True,
#                 trace=True,
#                 error_action="ignore",
#                 suppress_warnings=True,
#                 stepwise=True,n_fits=50)

In [ ]:
# # Import necessary libraries
# import matplotlib.pyplot as plt

# # Create a scatter plot of 'Product_Price' and 'Customer_Satisfaction'
# plt.figure(figsize=(10, 6))
# plt.scatter(maven_df['Product_Price'], maven_df['Customer_Satisfaction'], alpha=0.5)
# plt.title('Customer Satisfaction vs Product Price')
# plt.xlabel('Product Price')
# plt.ylabel('Customer Satisfaction')
# plt.show()


# # Calculate the total sales for each month
# monthly_sales = maven_df.groupby('YearMonth')['Customer_Satisfaction'].mean()
# # rolling_monthly_sales = maven_df.groupby('YearMonth')['Customer_Satisfaction'].rolling(3).mean()

# # Create a line plot of the total monthly sales
# plt.figure(figsize=(10, 6))
# monthly_sales.plot(kind='line')
# # rolling_monthly_sales.plot(kind='line')
# plt.title('Total Monthly Sales')
# plt.xlabel('Month')
# plt.ylabel('Total Sales')
# plt.show()

# maven_df.groupby(['YearMonth', 'Product_Name'])['Product_profit'].mean().sort_values(ascending=False).plot()


# # Create a line plot of the monthly sales for each product category
# plt.figure(figsize=(10, 6))
# sns.lineplot(data=category_sales, x='YearMonth', hue='Product_Category', palette='tab10')
# plt.title('Monthly Sales by Product Category')
# plt.xlabel('Month')
# plt.ylabel('Total Sales')
# plt.show()

# # Create a scatter plot of 'Customer_Satisfaction' and 'Units'
# plt.figure(figsize=(10, 6))
# plt.scatter(maven_df['Units'], maven_df['Customer_Satisfaction'], alpha=0.5)
# plt.title('Customer Satisfaction vs Units Sold')
# plt.xlabel('Units Sold')
# plt.ylabel('Customer Satisfaction')
# plt.show()

# # Calculate the monthly sales for each city
# city_sales = maven_df.groupby(['YearMonth', 'City'])['Customer_Satisfaction'].mean().unstack()

# # Create a line plot of the monthly sales for each city
# plt.figure(figsize=(10, 6))
# city_sales.plot(kind='line', ax=plt.gca())
# plt.title('Monthly Sales by City')
# plt.xlabel('Month')
# plt.ylabel('Total Sales')
# plt.show()